In [2]:
from pyforest import *

In [5]:
!pip install tensorflow

^C


In [8]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Flatten,Dropout,Conv2D,Input,Lambda
from keras.models import Model,Sequential
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from glob import glob

In [ ]:
img_size=[224,224]
train_path = 'Datasets/train'
valid_path = 'Datasets/test'
vgg=VGG16(input_shape=img_size+[3],weights='imagenet',include_top=False)

In [ ]:
for layer in vgg.layers:
    layer.trainable=False
folders = glob('Datasets/train/*')

In [ ]:
x=Flatten()(vgg.output)
prediction=Dense(len(folders),activation='softmax')(x)

In [ ]:
model=Model(inputs=vgg.input,outputs.prediction)

In [ ]:
model.compile(metrics=['accuracy'],loss='categorical_crossentropy',optimizer='adam')
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
r=model.fit_generator(training_set,validation_data=test_set,epochs=5,steps_per_epoch=len(training_set),validation_steps=len(test_set))

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
from keras.models import load_model

model.save('model_vgg16.h5')

In [ ]:
from IPython.display import display
from PIL import Image

In [ ]:
NeumonialPath="Datasets/val/PNEUMONIA/person1949_bacteria_4880.jpeg"
# NormalPath = "Datasets/val/NORMAL/NORMAL2-IM-1431-0001.jpeg"
display(Image.open(NeumonialPath))

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np


model = load_model('model_vgg16.h5')

img = image.load_img(NeumonialPath,target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
img_data = preprocess_input(x)
classes = model.predict(img_data)
print(classes)
if classes[0][0] > classes[0][1]:
    print('X-Ray image is NORMAL')
else:
    print('X-Ray image is Having NEUMONIAL')